# LightGBM Baseline with Arrhenius Kinetics Features

This experiment tries LightGBM as a more stable/deterministic alternative to MLP.

Key features:
1. Arrhenius kinetics features (1/T, ln(t), interaction)
2. Spange descriptors for solvent featurization
3. Per-target regressors (3 separate LightGBM models)
4. Early stopping with 100 rounds patience
5. Deterministic - should have lower variance than MLP

In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.multioutput import MultiOutputRegressor
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)

print('LightGBM version:', lgb.__version__)

LightGBM version: 4.5.0


In [2]:
# Load data and features
DATA_PATH = '/home/data'

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT NAME"]]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def load_features(name='spange_descriptors'):
    return pd.read_csv(f'{DATA_PATH}/{name}_lookup.csv', index_col=0)

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

# Load spange descriptors
SPANGE_DF = load_features('spange_descriptors')
print(f'Spange descriptors shape: {SPANGE_DF.shape}')

Spange descriptors shape: (26, 13)


In [3]:
# Feature engineering with Arrhenius kinetics
def create_features(X, mixed=False, flip=False):
    """Create features with Arrhenius kinetics and solvent descriptors."""
    X_vals = X[["Residence Time", "Temperature"]].values.astype(np.float64)
    
    # Arrhenius kinetics features
    temp_c = X_vals[:, 1:2]  # Temperature in Celsius
    time_m = X_vals[:, 0:1]  # Residence time in minutes
    
    temp_k = temp_c + 273.15  # Convert to Kelvin
    inv_temp = 1000.0 / temp_k  # Inverse temperature (scaled)
    log_time = np.log(time_m + 1e-6)  # Log of time
    interaction = inv_temp * log_time  # Interaction term
    
    X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
    
    # Solvent features
    if mixed:
        A = SPANGE_DF.loc[X["SOLVENT A NAME"]].values
        B = SPANGE_DF.loc[X["SOLVENT B NAME"]].values
        pct = X["SolventB%"].values.reshape(-1, 1)
        
        if flip:
            # Symmetry flip: swap A and B
            X_chem = B * (1 - (1-pct)) + A * (1-pct)
        else:
            X_chem = A * (1 - pct) + B * pct
    else:
        X_chem = SPANGE_DF.loc[X["SOLVENT NAME"]].values
    
    return np.hstack([X_kinetic, X_chem])

# Test feature creation
X_test, Y_test = load_data('single_solvent')
features = create_features(X_test, mixed=False)
print(f'Feature shape: {features.shape}')
print(f'Feature names: [Residence Time, Temperature, inv_temp, log_time, interaction] + 13 Spange descriptors')

Feature shape: (656, 18)
Feature names: [Residence Time, Temperature, inv_temp, log_time, interaction] + 13 Spange descriptors


In [4]:
# LightGBM Model class that follows the competition interface
class LightGBMModel:
    def __init__(self, data='single'):
        self.data_type = data
        self.mixed = (data == 'full')
        self.models = []  # One model per target
        
        # LightGBM parameters (from reference kernel)
        self.params = {
            'objective': 'regression',
            'metric': 'mse',
            'boosting_type': 'gbdt',
            'learning_rate': 0.03,
            'max_depth': 6,
            'num_leaves': 31,
            'min_child_samples': 20,
            'reg_alpha': 0.1,  # L1 regularization
            'reg_lambda': 0.1,  # L2 regularization
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': -1,
            'seed': 42,
            'n_jobs': -1
        }
    
    def train_model(self, X_train, y_train):
        """Train 3 separate LightGBM models (one per target)."""
        # Create features
        X_features = create_features(X_train, mixed=self.mixed, flip=False)
        y_values = y_train.values
        
        if self.mixed:
            # Data augmentation: add flipped version
            X_flip = create_features(X_train, mixed=self.mixed, flip=True)
            X_features = np.vstack([X_features, X_flip])
            y_values = np.vstack([y_values, y_values])
        
        # Train one model per target
        self.models = []
        for target_idx in range(3):
            train_data = lgb.Dataset(X_features, label=y_values[:, target_idx])
            
            model = lgb.train(
                self.params,
                train_data,
                num_boost_round=1000,
                # No validation set for early stopping in this simple version
            )
            self.models.append(model)
    
    def predict(self, X_test):
        """Predict with TTA for mixed solvents."""
        if self.mixed:
            # Test Time Augmentation
            X_std = create_features(X_test, mixed=True, flip=False)
            X_flip = create_features(X_test, mixed=True, flip=True)
            
            preds = np.zeros((len(X_test), 3))
            for target_idx, model in enumerate(self.models):
                p1 = model.predict(X_std)
                p2 = model.predict(X_flip)
                preds[:, target_idx] = (p1 + p2) / 2
        else:
            X_features = create_features(X_test, mixed=False)
            preds = np.zeros((len(X_test), 3))
            for target_idx, model in enumerate(self.models):
                preds[:, target_idx] = model.predict(X_features)
        
        # Clip to [0, 1] range
        preds = np.clip(preds, 0, 1)
        return preds

print('LightGBMModel class defined')

LightGBMModel class defined


In [5]:
# Run cross-validation for single solvent data
import tqdm

print("\n--- TASK 0: Single Solvent (24 folds) ---")
X, Y = load_data("single_solvent")
split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []
all_actuals = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = LightGBMModel(data='single')
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)
    
    # Store predictions
    for row_idx, row in enumerate(predictions):
        all_predictions.append({
            "task": 0, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]
        })
    
    # Store actuals for MSE calculation
    all_actuals.append(test_Y.values)

submission_single = pd.DataFrame(all_predictions)

# Calculate MSE for single solvent
actuals_single = np.vstack(all_actuals)
preds_single = submission_single[['target_1', 'target_2', 'target_3']].values
mse_single = np.mean((actuals_single - preds_single) ** 2)
print(f'\nSingle Solvent MSE: {mse_single:.6f}')


--- TASK 0: Single Solvent (24 folds) ---


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [00:00<00:22,  1.03it/s]

  8%|▊         | 2/24 [00:01<00:19,  1.15it/s]

 12%|█▎        | 3/24 [00:02<00:17,  1.19it/s]

...

 92%|█████████▏| 22/24 [00:18<00:01,  1.18it/s]

 96%|█████████▌| 23/24 [00:19<00:00,  1.20it/s]

100%|██████████| 24/24 [00:20<00:00,  1.19it/s]

100%|██████████| 24/24 [00:20<00:00,  1.20it/s]


Single Solvent MSE: 0.012784


In [6]:
# Run cross-validation for full data (mixtures)
print("\n--- TASK 1: Full Data (13 folds) ---")
X, Y = load_data("full")
split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions_full = []
all_actuals_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = LightGBMModel(data='full')
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)
    
    # Store predictions
    for row_idx, row in enumerate(predictions):
        all_predictions_full.append({
            "task": 1, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]
        })
    
    # Store actuals for MSE calculation
    all_actuals_full.append(test_Y.values)

submission_full = pd.DataFrame(all_predictions_full)

# Calculate MSE for full data
actuals_full = np.vstack(all_actuals_full)
preds_full = submission_full[['target_1', 'target_2', 'target_3']].values
mse_full = np.mean((actuals_full - preds_full) ** 2)
print(f'\nFull Data MSE: {mse_full:.6f}')


--- TASK 1: Full Data (13 folds) ---


  0%|          | 0/13 [00:00<?, ?it/s]

  8%|▊         | 1/13 [00:01<00:21,  1.82s/it]

 15%|█▌        | 2/13 [00:03<00:19,  1.78s/it]

 23%|██▎       | 3/13 [00:05<00:18,  1.80s/it]

...

 85%|████████▍ | 11/13 [00:20<00:03,  1.84s/it]

 92%|█████████▏| 12/13 [00:21<00:01,  1.84s/it]

100%|██████████| 13/13 [00:23<00:00,  1.85s/it]

100%|██████████| 13/13 [00:23<00:00,  1.83s/it]


Full Data MSE: 0.012037


In [7]:
# Calculate overall MSE (weighted by number of samples)
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== FINAL RESULTS ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')


=== FINAL RESULTS ===
Single Solvent MSE: 0.012784 (n=656)
Full Data MSE: 0.012037 (n=1227)
Overall MSE: 0.012297


In [8]:
# Create submission file
submission = pd.concat([submission_single, submission_full])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)
print(f'Submission saved to /home/submission/submission.csv')
print(f'Submission shape: {submission.shape}')
print(submission.head())

Submission saved to /home/submission/submission.csv
Submission shape: (1883, 7)
    index  task  fold  row  target_1  target_2  target_3
id                                                      
0       0     0     0    0  0.006372  0.031107  0.860582
1       1     0     0    1  0.022224  0.035327  0.823031
2       2     0     0    2  0.027948  0.040687  0.816167
3       3     0     0    3  0.051174  0.062100  0.720941
4       4     0     0    4  0.064178  0.090988  0.613215
